In [1]:
import unicodedata
import sys

def remove_accents(text):
    """
    Remove os acentos (diacríticos) de uma string usando a normalização Unicode.
    """
    nfkd_form = unicodedata.normalize('NFKD', text)
    return "".join([c for c in nfkd_form if not unicodedata.combining(c)])

def process_line(line):
    """
    Processa uma linha do arquivo:
    1. Remove os acentos.
    2. Se a linha tiver ao menos 88 caracteres, verifica o campo de variação
       (posições 86 a 88, índice 85 a 88 em Python). Se for '000', substitui por '019'.
    """
    # Remove acentos da linha inteira
    line = remove_accents(line)
    
    # Verifica se a linha possui comprimento suficiente para ter o campo de variação
    if len(line) >= 88:
        # Posições 86 a 88 (1-indexed) correspondem a índices 85 a 88 (exclusivo)
        if line[85:88] == "000":
            line = line[:85] + "019" + line[88:]
    return line

def process_file(input_filename, output_filename):
    """
    Lê o arquivo de entrada, processa cada linha e grava as alterações
    em um novo arquivo.
    """
    with open(input_filename, 'r', encoding='ISO8859_1') as fin, \
         open(output_filename, 'w', encoding='ISO8859_1') as fout:
        for line in fin:
            # Remove a quebra de linha e processa a linha
            processed_line = process_line(line.rstrip('\n'))
            fout.write(processed_line + '\n')


if len(sys.argv) != 3:
    print("Uso: python corrige_remessa.py <arquivo_entrada.rem>")
    sys.exit(1)

input_file = sys.argv[1]
print(sys.argv[1])
output_file = "C:\\Users\\Mateus\\Desktop\\cb250201-solved.rem"

process_file(input_file, output_file)
print(f"Arquivo processado. Saída em: {output_file}")

Uso: python corrige_remessa.py <arquivo_entrada.rem>


SystemExit: 1

d:\www\python\cadernos\venv\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
import pandas as pd

# 1. Carregar os arquivos Excel
df_vendas = pd.read_excel("/Users/mateusspinola/Desktop/Vendas-diária-31_03.xlsx", skiprows=6)
df_ver = pd.read_excel("/Users/mateusspinola/Documents/Comagro/Verificacao-pedidos-sec_vs_pls.xlsx")

# 2. Definir os nomes de colunas (ajuste conforme a verificação acima)
col_vendas_pedido = "Nº PED/ OS/ PRQ"
col_ver_pedido = "Nº Seculos"  # Ajuste também, se necessário

# 3. Criar o filtro e extrair os pedidos que não existem na planilha de verificação
df_nao_encontrados = df_vendas[
    ~df_vendas[col_vendas_pedido].isin(df_ver[col_ver_pedido])
]

# 4. Exibir ou salvar
print(df_nao_encontrados)

# 6. Se quiser salvar em Excel
df_nao_encontrados.to_excel("/Users/mateusspinola/Desktop/Pedidos_nao_encontrados.xlsx", index=False)

                    DATA  Nº PED/ OS/ PRQ  VALOR TOTAL       FORMA PAG  \
1    2025-03-31 00:00:00         143965.0       150.00          CARTAO   
6    2025-03-31 00:00:00         143974.0      1150.00          CARTAO   
18   2025-03-31 00:00:00         143892.0       667.08  DEPOSITO INTER   
20   2025-03-31 00:00:00         143920.0       443.00  DEPOSITO INTER   
22   2025-03-31 00:00:00         143890.0      4560.00  DEPOSITO INTER   
28   2025-03-31 00:00:00         143903.0       776.45  DEPOSITO INTER   
40   2025-03-31 00:00:00         143975.0       320.00  DEPOSITO INTER   
46   2025-03-31 00:00:00         143972.0      2200.00  DEPOSITO INTER   
53   2025-03-31 00:00:00         143971.0         2.00        DINHEIRO   
57   2025-03-31 00:00:00         143907.0         6.00        DINHEIRO   
58   2025-03-31 00:00:00         143889.0        13.00        DINHEIRO   
61   2025-03-31 00:00:00         143944.0       420.00          PEDIDO   
63   2025-03-31 00:00:00         14393

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Exemplo de conversão de arquivo de remessa CNAB400 para CNAB240
(versão ajustada para gerar registros de 240 posições conforme layout do BB).
Atenção: Os índices de corte dos campos (slice) no CNAB400 devem ser validados com o layout oficial.
Consulte o manual completo disponível em:
https://bb.com.br/docs/pub/emp/empl/dwn/000Completo.pdf
"""


def format_field(value, length, align='right', filler=' '):
    """
    Formata o campo para tamanho fixo.
      - align='left' usa ljust; se 'right' usa rjust.
      - filler é o caractere de preenchimento (ex.: '0' para numéricos ou ' ' para textos).
    """
    if not isinstance(value, str):
        value = str(value)
    if align == 'left':
        return value.ljust(length, filler)[:length]
    else:
        return value.rjust(length, filler)[:length]


def convert_header_cnab400_to_cnab240(header400):
    header240 = """00100000         214255350000103000164342001417000  0018800000001273795 COMAGRO PECAS E SERVICOS LTDA BANCO DO BRASIL                         10704202518042300132103000000                    00000000000000000000           CSP000            
00100011R01  020 2014255350000103000164342001417000  0018800000001273795 COMAGRO PE«AS E SERVI«OS LTDA                                                                                 000013241104202500000000                                 
"""
    return header240


def convert_detail_cnab400_to_cnab240(detail400, seq_num):
    # Conversão do registro de detalhe CNAB400 para os segmentos P e Q do CNAB240.
    # Os índices abaixo são exemplos. Ajuste-os conforme a real especificação do CNAB400.
    nosso_numero = detail400[40:55].strip()  # Ex.: pos. 41 a 55
    valor_titulo = detail400[100:115].strip()  # Ex.: pos. 101 a 115 (valor sem separador)
    # Segmento P:
    seg_p_fixed = (3+4+1+5+1+1+2+5+1+12+1+1+20+20+15)  # soma dos tamanhos dos campos já fixos
    # Calcula quantos caracteres faltam para completar 240
    remaining_p = 240 - seg_p_fixed
    seg_p = (
        format_field("001", 3, 'right', '0') +             # Código do Banco
        format_field("0001", 4, 'right', '0') +             # Lote (para este exemplo, fixo "0001")
        format_field("3", 1, 'left', ' ') +                 # Tipo de Registro = "3"
        format_field(str(seq_num).zfill(5), 5, 'right', '0') +  # Nº Sequencial do registro
        format_field("P", 1, 'left', ' ') +                 # Código do Segmento = P
        format_field(" ", 1, 'left', ' ') +                 # Uso Exclusivo
        format_field("01", 2, 'right', '0') +              # Código de Movimento Remessa
        format_field("00188", 5, 'right', '0') +           # Agência Mantenedora (exemplo)
        format_field("0", 1, 'left', ' ') +                # Dígito Agência
        format_field("000000127379", 12, 'right', '0') +    # Conta Corrente
        format_field("5", 1, 'left', ' ') +                # Dígito da Conta
        format_field(" ", 1, 'left', ' ') +                # Dígito Ver. Ag/Conta
        format_field(nosso_numero, 20, 'left', ' ') +       # Nosso Número
        format_field(" " * 20, 20, 'left', ' ') +          # Outros dados (placeholder)
        format_field(valor_titulo, 15, 'right', '0') +      # Valor Nominal do Título
        format_field(" " * remaining_p, remaining_p, 'left', ' ')
    )
    # Segmento Q: dados do sacado (exemplo fixo)
    seg_q_fixed = (3+4+1+5+1+1+2+1+14+40+40+15+8+3+15+2)
    remaining_q = 240 - seg_q_fixed
    seg_q = (
        format_field("001", 3, 'right', '0') +
        format_field("0001", 4, 'right', '0') +
        format_field("3", 1, 'left', ' ') +
        format_field(str(seq_num).zfill(5), 5, 'right', '0') +
        format_field("Q", 1, 'left', ' ') +
        format_field(" ", 1, 'left', ' ') +
        format_field("01", 2, 'right', '0') +
        format_field("2", 1, 'right', '0') +                # Tipo de Inscrição do Sacado
        format_field("08000000000000", 14, 'right', '0') +   # Número de Inscrição do Sacado
        format_field("NOME DO SACADO".ljust(40), 40, 'left', ' ') +
        format_field("ENDERECO DO SACADO".ljust(40), 40, 'left', ' ') +
        format_field("BAIRRO SACADO".ljust(15), 15, 'left', ' ') +
        format_field("00000000", 8, 'right', '0') +
        format_field("000", 3, 'right', '0') +
        format_field("CIDADE SACADO".ljust(15), 15, 'left', ' ') +
        format_field("UF", 2, 'left', ' ') +
        format_field(" " * remaining_q, remaining_q, 'left', ' ')
    )
    # Neste exemplo, não estamos gerando o segmento R (com descontos, protesto etc.).
    seg_r = ""
    # Garantir comprimento exato de 240 caracteres:
    seg_p = seg_p[:240]
    seg_q = seg_q[:240]
    return seg_p, seg_q, seg_r


def convert_trailer_cnab400_to_cnab240(trailer400):
    codigo_banco = trailer400[0:3]
    total_registros = trailer400[10:16]  # ajuste conforme o layout CNAB400
    trailer240 = (
        format_field(codigo_banco, 3, 'right', '0') +
        format_field("9999", 4, 'right', '0') +
        format_field("9", 1, 'left', ' ') +
        format_field(total_registros, 6, 'right', '0') +
        format_field(" " * (240 - (3+4+1+6)), 240 - (3+4+1+6), 'left', ' ')
    )
    assert len(trailer240) == 240, f"Tamanho do Trailer: {len(trailer240)} (esperado 240)"
    return trailer240


def main():
    input_file = "remessa_cnab400.rem"   # Nome do arquivo CNAB400 de entrada
    output_file = "remessa_cnab240.rem"   # Nome do arquivo CNAB240 a ser gerado

    with open(input_file, "r", encoding="latin1") as infile:
        linhas = infile.readlines()

    if not linhas:
        print("Arquivo de entrada vazio.")
        return

    cnab240_linhas = []
    # Converter o Header do Arquivo (primeira linha)
    header400 = linhas[0].rstrip("\n")
    cnab240_linhas.append(convert_header_cnab400_to_cnab240(header400))

    # Converter os registros de detalhe (linhas intermediárias)
    detalhe_cnab400 = linhas[1:-1]
    seq_num = 1
    for det in detalhe_cnab400:
        det = det.rstrip("\n")
        seg_p, seg_q, seg_r = convert_detail_cnab400_to_cnab240(det, seq_num)
        cnab240_linhas.append(seg_p)
        cnab240_linhas.append(seg_q)
        if seg_r:
            cnab240_linhas.append(seg_r)
        seq_num += 1

    # Converter o Trailer do Arquivo (última linha)
    trailer400 = linhas[-1].rstrip("\n")
    cnab240_linhas.append(convert_trailer_cnab400_to_cnab240(trailer400))

    with open(output_file, "w", encoding="latin1") as outfile:
        for linha in cnab240_linhas:
            outfile.write(linha + "\n")
    print(f"Arquivo CNAB240 gerado com sucesso: {output_file}")


if __name__ == "__main__":
    main()

Arquivo CNAB240 gerado com sucesso: remessa_cnab240.rem
